# Phase 4: Unsupervised Insights（无监督学习洞察）

---

## 📋 本阶段目标

通过无监督学习方法，从数据中**发现隐藏模式、识别异常点、提取新特征**，为监督学习提供新的视角和洞察。

### 核心任务

1. ✅ **聚类分析**：发现数据中的自然分组
   - K-Means聚类
   - DBSCAN（基于密度的聚类）
   - 层次聚类
2. ✅ **降维可视化**：在2D/3D空间中理解数据结构
   - PCA（主成分分析）
   - t-SNE（非线性降维）
3. ✅ **异常检测**：识别异常样本和离群点
   - Isolation Forest
   - Local Outlier Factor (LOF)
4. ✅ **特征提取**：从无监督学习中提取新特征
   - 聚类标签作为特征
   - 异常分数作为特征
5. ✅ **洞察整合**：将无监督学习的发现与监督学习结合

### 方法论

- **探索性分析**：不依赖标签，发现数据内在结构
- **多算法对比**：尝试不同算法，选择最佳方法
- **可视化驱动**：通过可视化理解聚类和降维结果
- **监督整合**：将无监督洞察反馈到监督学习

---

## 预期时间

⏱️ **20-60分钟**（取决于数据集大小）

---

## 1. 环境准备

In [ ]:
# 导入必要的库
import sys
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 导入本项目的模块
from src import data_preprocessing as dp
from src import unsupervised_pipeline as up
from src import supervised_pipeline as sp
from src import model_evaluation as me
from src import visualization as viz

# 设置
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 随机种子
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ 环境准备完成")
print(f"当前工作目录: {Path.cwd()}")

---

## 2. 数据加载与预处理

使用与Phase 2-3相同的数据，保持一致性

In [ ]:
# 加载数据
df = sns.load_dataset('titanic')
target_col = 'survived'

print(f"数据形状: {df.shape}")
print(f"目标变量: {target_col}")

# 查看数据
df.head()

In [ ]:
# 快速预处理（与Phase 2相同的方法）
print("="*60)
print("数据预处理")
print("="*60)

X, y = dp.quick_preprocess(
    df.copy(),
    target_col=target_col,
    drop_missing_threshold=0.5,
    handle_categorical=True,
    scale_features=True
)

print(f"\n处理后特征数: {X.shape[1]}")
print(f"处理后样本数: {X.shape[0]}")

# 保存特征名称
feature_names = X.columns.tolist()
print(f"\n特征列表（前10个）: {feature_names[:10]}")

---

## 3. 聚类分析

### 3.1 K-Means 聚类

使用肘部法则和轮廓系数自动确定最佳聚类数

In [ ]:
# K-Means聚类（自动确定K值）
print("="*60)
print("K-Means 聚类分析")
print("="*60)

kmeans_pipeline = up.ClusteringPipeline(
    method='kmeans',
    n_clusters=None,  # 自动确定
    random_state=RANDOM_STATE
)

kmeans_labels = kmeans_pipeline.fit_predict(X)

# 保存聚类标签
kmeans_k = kmeans_pipeline.n_clusters
print(f"\n最佳K值: {kmeans_k}")

In [ ]:
# 分析聚类结果与目标变量的关系
print("\n" + "="*60)
print("K-Means聚类 vs 实际标签（survived）")
print("="*60)

cluster_analysis = pd.DataFrame({
    'cluster': kmeans_labels,
    'survived': y
})

# 交叉表
crosstab = pd.crosstab(
    cluster_analysis['cluster'],
    cluster_analysis['survived'],
    normalize='index'
)

print("\n各聚类中的生存率:")
print(crosstab)

# 可视化
crosstab.plot(kind='bar', stacked=False, figsize=(10, 6), colormap='viridis')
plt.title('K-Means聚类 vs 生存率')
plt.xlabel('聚类标签')
plt.ylabel('比例')
plt.legend(['未生存', '生存'])
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# 发现洞察
print("\n💡 洞察:")
for cluster_id in range(kmeans_k):
    survival_rate = crosstab.loc[cluster_id, 1] if 1 in crosstab.columns else 0
    count = (kmeans_labels == cluster_id).sum()
    print(f"  聚类 {cluster_id}: {count}个样本, 生存率 {survival_rate:.1%}")

### 3.2 DBSCAN 聚类（基于密度）

DBSCAN可以自动发现任意形状的簇，并识别噪声点

In [ ]:
# DBSCAN聚类
print("\n" + "="*60)
print("DBSCAN 聚类分析")
print("="*60)

dbscan_pipeline = up.ClusteringPipeline(
    method='dbscan',
    random_state=RANDOM_STATE
)

dbscan_labels = dbscan_pipeline.fit_predict(X)

# 统计噪声点
n_noise = (dbscan_labels == -1).sum()
n_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)

print(f"\n发现 {n_clusters} 个聚类")
print(f"噪声点数量: {n_noise} ({n_noise/len(dbscan_labels)*100:.1f}%)")

### 3.3 层次聚类

层次聚类构建聚类树，可以观察不同层次的分组

In [ ]:
# 层次聚类
print("\n" + "="*60)
print("层次聚类分析")
print("="*60)

hierarchical_pipeline = up.ClusteringPipeline(
    method='hierarchical',
    n_clusters=kmeans_k,  # 使用与K-Means相同的聚类数
    random_state=RANDOM_STATE
)

hierarchical_labels = hierarchical_pipeline.fit_predict(X)

---

## 4. 降维与可视化

### 4.1 PCA降维

PCA是线性降维方法，保留最大方差

In [ ]:
# PCA降维到2D
print("="*60)
print("PCA 降维分析")
print("="*60)

pca_pipeline = up.DimensionReductionPipeline(
    method='pca',
    n_components=2,
    random_state=RANDOM_STATE
)

X_pca = pca_pipeline.fit_transform(X)

print(f"\n降维后形状: {X_pca.shape}")

In [ ]:
# 可视化PCA结果（按真实标签着色）
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 左图：按真实标签着色
scatter1 = axes[0].scatter(
    X_pca[:, 0],
    X_pca[:, 1],
    c=y,
    cmap='RdYlGn',
    alpha=0.6,
    edgecolors='k',
    s=50
)
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')
axes[0].set_title('PCA降维 - 按实际标签（Survived）着色')
axes[0].grid(alpha=0.3)
plt.colorbar(scatter1, ax=axes[0], label='Survived')

# 右图：按K-Means聚类着色
scatter2 = axes[1].scatter(
    X_pca[:, 0],
    X_pca[:, 1],
    c=kmeans_labels,
    cmap='tab10',
    alpha=0.6,
    edgecolors='k',
    s=50
)
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC2')
axes[1].set_title('PCA降维 - 按K-Means聚类着色')
axes[1].grid(alpha=0.3)
plt.colorbar(scatter2, ax=axes[1], label='Cluster')

plt.tight_layout()
plt.show()

print("\n💡 观察: 对比两图，看聚类是否与真实标签对应")

### 4.2 t-SNE降维

t-SNE是非线性降维方法，善于保留局部结构

In [ ]:
# t-SNE降维
print("\n" + "="*60)
print("t-SNE 降维分析")
print("="*60)
print("⏳ t-SNE计算较慢，请稍候...")

tsne_pipeline = up.DimensionReductionPipeline(
    method='tsne',
    n_components=2,
    random_state=RANDOM_STATE
)

X_tsne = tsne_pipeline.fit_transform(X)

print(f"\n降维后形状: {X_tsne.shape}")
print("✅ t-SNE完成")

In [ ]:
# 可视化t-SNE结果
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 左图：按真实标签着色
scatter1 = axes[0].scatter(
    X_tsne[:, 0],
    X_tsne[:, 1],
    c=y,
    cmap='RdYlGn',
    alpha=0.6,
    edgecolors='k',
    s=50
)
axes[0].set_xlabel('t-SNE 1')
axes[0].set_ylabel('t-SNE 2')
axes[0].set_title('t-SNE降维 - 按实际标签（Survived）着色')
axes[0].grid(alpha=0.3)
plt.colorbar(scatter1, ax=axes[0], label='Survived')

# 右图：按K-Means聚类着色
scatter2 = axes[1].scatter(
    X_tsne[:, 0],
    X_tsne[:, 1],
    c=kmeans_labels,
    cmap='tab10',
    alpha=0.6,
    edgecolors='k',
    s=50
)
axes[1].set_xlabel('t-SNE 1')
axes[1].set_ylabel('t-SNE 2')
axes[1].set_title('t-SNE降维 - 按K-Means聚类着色')
axes[1].grid(alpha=0.3)
plt.colorbar(scatter2, ax=axes[1], label='Cluster')

plt.tight_layout()
plt.show()

print("\n💡 对比: t-SNE vs PCA")
print("  - PCA: 线性降维，保留全局结构")
print("  - t-SNE: 非线性降维，保留局部结构，簇更明显")

---

## 5. 异常检测

### 5.1 Isolation Forest 异常检测

In [ ]:
# Isolation Forest异常检测
print("="*60)
print("Isolation Forest 异常检测")
print("="*60)

iforest_pipeline = up.AnomalyDetectionPipeline(
    method='isolation_forest',
    contamination=0.1,  # 假设10%的数据是异常
    random_state=RANDOM_STATE
)

iforest_predictions = iforest_pipeline.fit_predict(X)

# 获取异常分数
iforest_scores = iforest_pipeline.get_anomaly_scores(X)

# 标记异常点
is_anomaly_iforest = iforest_predictions == -1

print(f"\n异常样本索引（前10个）: {np.where(is_anomaly_iforest)[0][:10].tolist()}")

In [ ]:
# 可视化异常点（在PCA降维空间中）
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 左图：PCA空间中的异常点
axes[0].scatter(
    X_pca[~is_anomaly_iforest, 0],
    X_pca[~is_anomaly_iforest, 1],
    c='blue',
    alpha=0.5,
    label='正常',
    s=30
)
axes[0].scatter(
    X_pca[is_anomaly_iforest, 0],
    X_pca[is_anomaly_iforest, 1],
    c='red',
    alpha=0.8,
    label='异常',
    s=100,
    marker='x',
    linewidths=2
)
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')
axes[0].set_title('Isolation Forest - PCA空间中的异常点')
axes[0].legend()
axes[0].grid(alpha=0.3)

# 右图：异常分数分布
axes[1].hist(iforest_scores[~is_anomaly_iforest], bins=50, alpha=0.6, label='正常', color='blue')
axes[1].hist(iforest_scores[is_anomaly_iforest], bins=20, alpha=0.6, label='异常', color='red')
axes[1].set_xlabel('异常分数')
axes[1].set_ylabel('样本数')
axes[1].set_title('异常分数分布')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 5.2 Local Outlier Factor (LOF) 异常检测

In [ ]:
# LOF异常检测
print("\n" + "="*60)
print("LOF (Local Outlier Factor) 异常检测")
print("="*60)

lof_pipeline = up.AnomalyDetectionPipeline(
    method='lof',
    contamination=0.1,
    random_state=RANDOM_STATE
)

lof_predictions = lof_pipeline.fit_predict(X)

is_anomaly_lof = lof_predictions == -1

In [ ]:
# 对比两种异常检测方法
print("\n" + "="*60)
print("异常检测方法对比")
print("="*60)

comparison = pd.DataFrame({
    'Isolation Forest': is_anomaly_iforest.astype(int),
    'LOF': is_anomaly_lof.astype(int)
})

# 交叉表
crosstab_anomaly = pd.crosstab(
    comparison['Isolation Forest'],
    comparison['LOF'],
    rownames=['IForest'],
    colnames=['LOF']
)

print("\n两种方法检测结果对比（0=正常, 1=异常）:")
print(crosstab_anomaly)

# 计算一致性
agreement = (is_anomaly_iforest == is_anomaly_lof).sum() / len(X) * 100
both_anomaly = (is_anomaly_iforest & is_anomaly_lof).sum()

print(f"\n两种方法一致性: {agreement:.1f}%")
print(f"两种方法都认为是异常的样本数: {both_anomaly}")

### 5.3 异常点分析

In [ ]:
# 分析异常点的特征
print("\n" + "="*60)
print("异常点特征分析")
print("="*60)

# 选择两种方法都认为是异常的点
strong_anomalies = is_anomaly_iforest & is_anomaly_lof

if strong_anomalies.sum() > 0:
    print(f"\n强异常点数量: {strong_anomalies.sum()}")
    
    # 查看这些异常点在原始数据中的情况
    anomaly_df = df.iloc[np.where(strong_anomalies)[0]]
    
    print("\n异常点样本（前5个）:")
    print(anomaly_df.head())
    
    # 异常点的目标变量分布
    if target_col in anomaly_df.columns:
        print(f"\n异常点中的生存率: {anomaly_df[target_col].mean():.2%}")
        print(f"全体数据的生存率: {df[target_col].mean():.2%}")
else:
    print("\n没有两种方法都认为是异常的点")

---

## 6. 无监督学习特征提取

将无监督学习的发现转化为新特征，用于监督学习

In [ ]:
# 创建无监督学习特征
print("="*60)
print("从无监督学习中提取新特征")
print("="*60)

# 原始特征
X_enhanced = X.copy()

# 1. 添加聚类标签作为特征
X_enhanced['cluster_kmeans'] = kmeans_labels
X_enhanced['cluster_hierarchical'] = hierarchical_labels

# 2. 添加异常分数作为特征
X_enhanced['anomaly_score_iforest'] = iforest_scores

# 3. 添加PCA主成分作为特征
X_enhanced['pca_1'] = X_pca[:, 0]
X_enhanced['pca_2'] = X_pca[:, 1]

# 4. 添加异常标记
X_enhanced['is_anomaly'] = is_anomaly_iforest.astype(int)

print(f"\n原始特征数: {X.shape[1]}")
print(f"增强后特征数: {X_enhanced.shape[1]}")
print(f"新增特征数: {X_enhanced.shape[1] - X.shape[1]}")

print("\n新增特征列表:")
new_features = [col for col in X_enhanced.columns if col not in X.columns]
for feat in new_features:
    print(f"  - {feat}")

---

## 7. 无监督特征对监督学习的影响

评估添加无监督学习特征后，监督学习性能是否提升

In [ ]:
# 划分数据集
from sklearn.model_selection import train_test_split

# 基线：不包含无监督特征
X_train_base, X_test_base, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=RANDOM_STATE,
    stratify=y
)

# 增强：包含无监督特征
X_train_enhanced, X_test_enhanced, _, _ = train_test_split(
    X_enhanced, y,
    test_size=0.3,
    random_state=RANDOM_STATE,
    stratify=y
)

print("数据集划分:")
print(f"  基线训练集: {X_train_base.shape}")
print(f"  增强训练集: {X_train_enhanced.shape}")
print(f"  测试集: {X_test_base.shape}")

In [ ]:
# 训练基线模型（无无监督特征）
print("\n" + "="*60)
print("训练基线模型（无无监督特征）")
print("="*60)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

model_base = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
model_base.fit(X_train_base, y_train)

y_pred_base = model_base.predict(X_test_base)
acc_base = accuracy_score(y_test, y_pred_base)
f1_base = f1_score(y_test, y_pred_base, average='binary')

print(f"\n基线模型性能:")
print(f"  Accuracy: {acc_base:.4f}")
print(f"  F1-Score: {f1_base:.4f}")

In [ ]:
# 训练增强模型（包含无监督特征）
print("\n" + "="*60)
print("训练增强模型（包含无监督特征）")
print("="*60)

model_enhanced = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
model_enhanced.fit(X_train_enhanced, y_train)

y_pred_enhanced = model_enhanced.predict(X_test_enhanced)
acc_enhanced = accuracy_score(y_test, y_pred_enhanced)
f1_enhanced = f1_score(y_test, y_pred_enhanced, average='binary')

print(f"\n增强模型性能:")
print(f"  Accuracy: {acc_enhanced:.4f}")
print(f"  F1-Score: {f1_enhanced:.4f}")

In [ ]:
# 性能对比
print("\n" + "="*60)
print("无监督特征对监督学习的影响")
print("="*60)

comparison_df = pd.DataFrame({
    '模型': ['基线（无无监督特征）', '增强（含无监督特征）'],
    'Accuracy': [acc_base, acc_enhanced],
    'F1-Score': [f1_base, f1_enhanced],
    '特征数': [X.shape[1], X_enhanced.shape[1]]
})

print("\n")
print(comparison_df.to_string(index=False))

# 计算提升
acc_improvement = (acc_enhanced - acc_base) / acc_base * 100
f1_improvement = (f1_enhanced - f1_base) / f1_base * 100

print(f"\n📈 性能变化:")
print(f"  Accuracy: {acc_improvement:+.2f}%")
print(f"  F1-Score: {f1_improvement:+.2f}%")

if acc_improvement > 1:
    print("\n✅ 无监督特征显著提升了模型性能！")
elif acc_improvement > 0:
    print("\n✓  无监督特征略微提升了模型性能")
else:
    print("\n⚠️  无监督特征未能提升性能，可能原因:")
    print("   - 聚类未能发现有用的模式")
    print("   - 特征冗余导致过拟合")
    print("   - 需要特征选择")

In [ ]:
# 可视化性能对比
fig, ax = plt.subplots(figsize=(10, 6))

x_pos = np.arange(2)
width = 0.35

ax.bar(x_pos - width/2, [acc_base, f1_base], width, label='基线模型', color='skyblue', alpha=0.8)
ax.bar(x_pos + width/2, [acc_enhanced, f1_enhanced], width, label='增强模型', color='orange', alpha=0.8)

ax.set_ylabel('得分')
ax.set_title('无监督特征对监督学习性能的影响')
ax.set_xticks(x_pos)
ax.set_xticklabels(['Accuracy', 'F1-Score'])
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.set_ylim([0, 1])

# 添加数值标签
for i, (v1, v2) in enumerate(zip([acc_base, f1_base], [acc_enhanced, f1_enhanced])):
    ax.text(i - width/2, v1 + 0.02, f'{v1:.3f}', ha='center', fontsize=10)
    ax.text(i + width/2, v2 + 0.02, f'{v2:.3f}', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

### 7.1 特征重要性分析（增强模型）

In [ ]:
# 查看无监督特征的重要性
print("="*60)
print("特征重要性分析")
print("="*60)

# 获取特征重要性
feature_importance = pd.DataFrame({
    'feature': X_enhanced.columns,
    'importance': model_enhanced.feature_importances_
}).sort_values('importance', ascending=False)

# 标记无监督特征
feature_importance['type'] = feature_importance['feature'].apply(
    lambda x: 'Unsupervised' if x in new_features else 'Original'
)

print("\nTop 20 最重要特征:")
print(feature_importance.head(20).to_string(index=False))

# 统计无监督特征在Top 20中的占比
top20 = feature_importance.head(20)
unsup_in_top20 = (top20['type'] == 'Unsupervised').sum()

print(f"\n💡 洞察:")
print(f"  无监督特征在Top 20中的数量: {unsup_in_top20}/{len(new_features)}")
print(f"  无监督特征平均重要性: {feature_importance[feature_importance['type']=='Unsupervised']['importance'].mean():.4f}")
print(f"  原始特征平均重要性: {feature_importance[feature_importance['type']=='Original']['importance'].mean():.4f}")

In [ ]:
# 可视化特征重要性
top_features = feature_importance.head(15)

fig, ax = plt.subplots(figsize=(10, 8))

colors = ['orange' if t == 'Unsupervised' else 'skyblue' for t in top_features['type']]

ax.barh(range(len(top_features)), top_features['importance'], color=colors, alpha=0.7)
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features['feature'])
ax.set_xlabel('重要性')
ax.set_title('Top 15 特征重要性（橙色=无监督特征）')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

---

## 8. Phase 4 关键洞察总结

In [ ]:
print("\n" + "="*60)
print("📊 Phase 4: Unsupervised Insights 总结")
print("="*60)

print("\n1. 聚类分析")
print(f"   ✅ K-Means最佳聚类数: {kmeans_k}")
print(f"   ✅ DBSCAN发现聚类数: {n_clusters}")
print(f"   ✅ DBSCAN噪声点比例: {n_noise/len(X)*100:.1f}%")

print("\n2. 降维与可视化")
print(f"   ✅ PCA累计解释方差: {pca_pipeline.model.explained_variance_ratio_.sum():.1%}")
print("   ✅ t-SNE成功可视化数据分布")

print("\n3. 异常检测")
print(f"   ✅ Isolation Forest检测异常: {is_anomaly_iforest.sum()}个")
print(f"   ✅ LOF检测异常: {is_anomaly_lof.sum()}个")
print(f"   ✅ 两种方法一致性: {agreement:.1f}%")

print("\n4. 特征提取")
print(f"   ✅ 从无监督学习提取 {len(new_features)} 个新特征")
print(f"   ✅ 特征数量提升: {X.shape[1]} → {X_enhanced.shape[1]}")

print("\n5. 监督学习性能影响")
print(f"   {'✅' if acc_improvement > 0 else '⚠️ '} Accuracy变化: {acc_improvement:+.2f}%")
print(f"   {'✅' if f1_improvement > 0 else '⚠️ '} F1-Score变化: {f1_improvement:+.2f}%")

print("\n" + "="*60)
print("🔍 关键发现")
print("="*60)

insights = []

# 聚类洞察
if kmeans_k <= 3:
    insights.append(f"数据可能存在 {kmeans_k} 个明显的自然分组")

# 异常检测洞察
if strong_anomalies.sum() > 0:
    insights.append(f"发现 {strong_anomalies.sum()} 个高置信度异常样本")

# 性能洞察
if acc_improvement > 2:
    insights.append("无监督特征显著提升了监督学习性能（>2%）")
elif acc_improvement > 0:
    insights.append("无监督特征对监督学习有轻微帮助")
else:
    insights.append("无监督特征未能改善性能，建议特征选择")

# 特征重要性洞察
if unsup_in_top20 > 0:
    insights.append(f"{unsup_in_top20} 个无监督特征进入Top 20重要特征")

for i, insight in enumerate(insights, 1):
    print(f"  {i}. {insight}")

### 8.1 Phase 4 检查清单

In [ ]:
print("\n" + "="*60)
print("✅ Phase 4 完成检查清单")
print("="*60)

checklist = [
    "✅ K-Means聚类分析完成",
    "✅ DBSCAN聚类分析完成",
    "✅ 层次聚类分析完成",
    "✅ PCA降维可视化完成",
    "✅ t-SNE降维可视化完成",
    "✅ Isolation Forest异常检测完成",
    "✅ LOF异常检测完成",
    "✅ 异常点分析完成",
    "✅ 无监督特征提取完成",
    "✅ 监督学习性能对比完成",
    "✅ 特征重要性分析完成",
    "✅ 关键洞察总结完成"
]

for item in checklist:
    print(f"  {item}")

print("\n" + "="*60)
print("🎉 Phase 4: Unsupervised Insights 完成！")
print("="*60)
print("\n下一步选项:")
print("  1. 前往 Phase 5: 整合监督与无监督方法")
print("  2. 前往 Phase 6: 最终解决方案")
print("  3. 回顾并优化 Phase 1-4 的发现")

---

## 9. 保存结果（可选）

In [ ]:
# 保存聚类结果
# results_df = X.copy()
# results_df['cluster_kmeans'] = kmeans_labels
# results_df['is_anomaly'] = is_anomaly_iforest
# results_df['target'] = y

# results_path = Path('results/phase4_unsupervised_results.csv')
# results_path.parent.mkdir(parents=True, exist_ok=True)
# results_df.to_csv(results_path, index=False)
# print(f"✅ 结果已保存到: {results_path}")

# 保存增强特征
# features_path = Path('data/processed/phase4_enhanced_features.csv')
# features_path.parent.mkdir(parents=True, exist_ok=True)
# X_enhanced.to_csv(features_path, index=False)
# print(f"✅ 增强特征已保存到: {features_path}")